In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

In [2]:
text = "en un lugar de la mancha de cuyo nombre no quiero acordarme"
chars = list(set(text))
char2idx = {ch: idx for idx, ch in enumerate(chars)}
idx2char = {idx: ch for ch, idx in char2idx.items()}

In [3]:
seq_length = 10
step = 1
X, y = [], []
for i in range(0, len(text) - seq_length, step):
    X.append([char2idx[c] for c in text[i:i+seq_length]])
    y.append(char2idx[text[i+seq_length]])

X = torch.tensor(X)
y = torch.tensor(y)

In [4]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

In [5]:
model = CharRNN(len(chars), 128)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [6]:
for epoch in range(100):
    out = model(X)
    loss = loss_fn(out, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 2.8420
Epoch 20, Loss: 0.0118
Epoch 40, Loss: 0.0007
Epoch 60, Loss: 0.0003
Epoch 80, Loss: 0.0003


In [7]:
def generate(model, start, length):
    model.eval()
    result = start
    input_seq = torch.tensor([[char2idx[c] for c in start]])
    for _ in range(length):
        output = model(input_seq)
        idx = torch.argmax(output, dim=1)[-1].item()
        result += idx2char[idx]
        input_seq = torch.cat([input_seq[:, 1:], torch.tensor([[idx]])], dim=1)
    return result

In [8]:
print(generate(model, "en un luga", 50))

en un lugar de la mancha de cuyo nombre no quiero acordarmer


In [9]:
class CharLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [10]:
model_lstm = CharLSTM(len(chars), 128)
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.01)

In [11]:
for epoch in range(100):
    out = model_lstm(X)
    loss = loss_fn(out, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print(f"LSTM Epoch {epoch}, Loss: {loss.item():.4f}")

LSTM Epoch 0, Loss: 2.8431
LSTM Epoch 20, Loss: 0.0044
LSTM Epoch 40, Loss: 0.0005
LSTM Epoch 60, Loss: 0.0003
LSTM Epoch 80, Loss: 0.0002


In [12]:
print(generate(model_lstm, "en un luga", 50))

en un lugar de la mancha de cuyo nombre no quiero acordarme 


In [13]:
pip install --upgrade transformers


In [14]:
from transformers import pipeline

In [29]:
#generator = pipeline("text-generation", model="gpt2")
generator = pipeline(
    "text-generation",
    model="datificate/gpt2-small-spanish"
)

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
#result = generator(
#    "En un lugar de la Mancha,",
#    max_new_tokens=100,
#    temperature=0.9,
#    top_k=50,
#    repetition_penalty=1.2,
#    do_sample=True,
#    num_return_sequences=1
#)

In [31]:
result = generator(
    "En un lugar de la Mancha,",
    max_new_tokens=150,
    temperature=0.7,       # Menos aleatorio, más coherente
    top_k=40,              # Un poco menos de variedad
    top_p=0.92,            # Nucleus sampling
    repetition_penalty=1.15, # Evita repeticiones
    do_sample=True,
    num_return_sequences=3  # Te da 3 opciones para comparar
)
print(result[0]['generated_text'])


En un lugar de la Mancha, la ciudad se encuentra situada a una altitud media entre los 700 y los 900 msnm.

El clima es mediterráneo continentalizado con influencia de los vientos alisios del oeste en dirección al este y sudoeste (este-oeste) formando un clima templado que se caracteriza por las temperaturas más bajas durante el invierno estival, así como abundantes precipitaciones anuales en verano e inviernos cortos debido principalmente a la poca humedad relativa; también son frecuentes las heladas invernales, cuando las temperaturas son elevadas considerablemente hasta alcanzar valores superiores a los 1º C. En cuanto a la precipitación pluvial anual es de unos 2.000 mm., mientras que en las lluvias se da entre los 700 y los 900 mm.; además de ser muy común en zonas montañosas, especialmente en los valles de los ríos Tajo
